In [1]:
import pandas as pd
import json
import http.client

#Get dataset containing Postcode data
dummyPostCodeData = pd.read_csv('dummy_patientdata.csv')

#Create Valid PostCode dataset
postCodes = dummyPostCodeData
postCodes = postCodes[['Address_Line4']]
postCodes['PostCode'] = postCodes['Address_Line4'].str.replace(' ','',regex= True)

postCodes.dropna()
postCodes = postCodes[(postCodes['PostCode'].str.len() >= 6) & (postCodes['PostCode'].str.len() <= 8)]
postCodes['PostCode'] = postCodes['PostCode'].str.upper()
postCodes = postCodes.drop_duplicates()


C:\Users\DeepakTomar\AppData\Local\Temp\ipykernel_14952\1906382098.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  postCodes['PostCode'] = postCodes['Address_Line4'].str.replace(' ','',regex= True)
C:\Users\DeepakTomar\AppData\Local\Temp\ipykernel_14952\1906382098.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  postCodes['PostCode'] = postCodes['PostCode'].str.upper()


In [2]:
#Create 1km*1km x,y coordinates
roundOffCoord=pd.Series(range(500,9999999,500))
roundOffCoord = pd.DataFrame(roundOffCoord, columns={'StartCoordinates'})
roundOffCoord['EndCoordinates'] = roundOffCoord['StartCoordinates'].apply(lambda x : x + 499)
roundOffCoord['Value'] = roundOffCoord['StartCoordinates'].apply(lambda x : x + 500 if x % 1000 == 0 else x)

print(roundOffCoord)

       StartCoordinates  EndCoordinates    Value
0                   500             999      500
1                  1000            1499     1500
2                  1500            1999     1500
3                  2000            2499     2500
4                  2500            2999     2500
...                 ...             ...      ...
19994           9997500         9997999  9997500
19995           9998000         9998499  9998500
19996           9998500         9998999  9998500
19997           9999000         9999499  9999500
19998           9999500         9999999  9999500

[19999 rows x 3 columns]


In [3]:
#Get x,y coordinates for the valid postcodes using API call
import requests

cols = ['PostCode', 'PostCodeArea', 'PostcodeSector', 'Easting', 'Northing']
    
gridData = pd.DataFrame(columns=cols)
for code in postCodes['PostCode']:
    
    #Make an API call for each postcode from postCodes dataset to get x and y coordinates
    response = requests.get(f'http://api.getthedata.com/postcode/{code}')
    output = response.json()
    
    arr = []
    #Check if the postcode is a valid postcode i.e. a match.
    if output['status'] == 'match':
            arr = [{'PostCode' : output['data']['postcode'],'PostCodeArea' : output['data']['postcode_area']
                ,'PostcodeSector': output['data']['postcode_sector'],'Easting_Actual' : output['data']['easting']
                ,'Northing_Actual' : output['data']['northing']}]
            
            newdata = pd.DataFrame(arr)
            newdata = pd.merge(newdata, roundOffCoord,  how='cross')
            
            #Group easting coordinates into 1*1km area
            xnewdata = newdata.query("Easting_Actual >= StartCoordinates and Easting_Actual <= EndCoordinates" )
            xnewdata['Easting'] = xnewdata['Value']
            xnewdata = xnewdata[['PostCode', 'PostCodeArea', 'PostcodeSector', 'Easting_Actual', 'Easting']]
            
            #Group northing coordinates into 1*1km area
            ynewdata = newdata.query("Northing_Actual >= StartCoordinates and Northing_Actual <= EndCoordinates" )
            ynewdata['Northing'] = ynewdata['Value']
            ynewdata = ynewdata[['PostCode', 'Northing_Actual', 'Northing']]
            
            #Combine easting and northing data together
            combinedcoord = xnewdata.merge(ynewdata, on='PostCode')
            combinedcoord = combinedcoord[['PostCode', 'PostCodeArea', 'PostcodeSector', 'Easting_Actual', 'Easting', 'Northing_Actual', 'Northing']]
            combinedcoord = combinedcoord.drop_duplicates()
            
            #Add each postcode data into one dataset
            gridData = gridData.append(combinedcoord, ignore_index=True)
    response.close()
    
#Get rid of any duplicates
gridData = gridData.drop_duplicates()      


C:\Users\DeepakTomar\AppData\Local\Temp\ipykernel_14952\3593143546.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xnewdata['Easting'] = xnewdata['Value']
C:\Users\DeepakTomar\AppData\Local\Temp\ipykernel_14952\3593143546.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ynewdata['Northing'] = ynewdata['Value']
C:\Users\DeepakTomar\AppData\Local\Temp\ipykernel_14952\3593143546.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

In [4]:
#Get air pollution data
airPoll = pd.read_csv('mappm102021g.csv')

#Data begins from row 6, so skip first 5 rows and using 6th row as column header
airPoll = airPoll.rename(columns=airPoll.iloc[4]).drop(airPoll.index[0:5])

#Create a lookup dataset for air pollution and postcodes
airPoll['x'] = airPoll['x'].astype('int')
airPoll['y'] = airPoll['y'].astype('int')
airPollLookUp = pd.merge(airPoll, gridData, how='cross')
airPollLookUp = airPollLookUp.query(" x == Easting and y == Northing")

print(airPollLookUp)

C:\Users\DeepakTomar\AppData\Local\Temp\ipykernel_14952\4078738107.py:2: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  airPoll = pd.read_csv('mappm102021g.csv')


       gridcode       x       y pm102021g  PostCode PostCodeArea  \
314920   430208  327500  676500  9.271426   EH6 6SG           EH   
314921   430208  327500  676500  9.271426   EH6 7BG           EH   
317104   431590  329500  674500    7.9829  EH15 1ED           EH   
319418   432972  331500  672500  8.393893  EH15 2RP           EH   
327567   437724  253500  665500  9.556187   G51 4TF            G   

       PostcodeSector Easting Northing  Easting_Actual  Northing_Actual  
314920          EH6 6  327500   676500        327142.0         676467.0  
314921          EH6 7  327500   676500        327282.0         676375.0  
317104         EH15 1  329500   674500        329820.0         674795.0  
319418         EH15 2  331500   672500        331540.0         672288.0  
327567          G51 4  253500   665500        253769.0         665746.0  
